In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import datetime
import itertools
from sklearn import preprocessing
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from catboost import Pool
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_Columns', 100)

In [2]:
train_y = pd.read_csv('../input/train.csv')
print(train_y.shape)
train_X = pd.read_csv('../input/train_X.csv')
print(train_X.shape)
test_X = pd.read_csv('../input/test_X.csv')
print(test_X.shape)

(12026, 19)
(12026, 371)
(12008, 372)


In [3]:
n_fold = 5

In [4]:
target = train_y['likes'].apply(np.log1p)

In [5]:
train_X.head()

,description_w2v_0,description_w2v_1,description_w2v_2,description_w2v_3,description_w2v_4,description_w2v_5,description_w2v_6,description_w2v_7,description_w2v_8,description_w2v_9,description_w2v_10,description_w2v_11,description_w2v_12,description_w2v_13,description_w2v_14,description_w2v_15,description_w2v_16,description_w2v_17,description_w2v_18,description_w2v_19,description_w2v_20,description_w2v_21,description_w2v_22,description_w2v_23,description_w2v_24,description_w2v_25,description_w2v_26,description_w2v_27,description_w2v_28,description_w2v_29,description_w2v_30,description_w2v_31,description_w2v_32,description_w2v_33,description_w2v_34,description_w2v_35,description_w2v_36,description_w2v_37,description_w2v_38,description_w2v_39,description_w2v_40,description_w2v_41,description_w2v_42,description_w2v_43,description_w2v_44,description_w2v_45,description_w2v_46,description_w2v_47,description_w2v_48,description_w2v_49,...,PC_United States,PC_Norway,PC_Japan,PC_Switzerland,PC_India,PC_Austria,PC_Spain,PC_China,TC_etching,TC_engraving,TC_albumen print,TC_gelatin silver print,TC_letterpress printing,TC_drypoint,TC_salted paper print,TC_slide,TC_painting,TC_steel engraving,HP_0,HP_1,HP_2,color_R,color_G,color_B,color_H,color_S,color_V,plt_R,plt_G,plt_B,plt_H,plt_S,plt_V,color_count,palette_count,CL_color_b_color_g,CL_color_g_color_r,CL_color_r_color_b,CL_sum,PL_color_b_color_g,PL_color_g_color_r,PL_color_r_color_b,PL_sum,occupation_count,principal_maker_count,birth_year,death_year,maker_death_old,TE_principal_maker,TE_principal_or_first_maker
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.297882,0.262706,0.191490,0.111562,0.357162,0.297882,0.316709,0.205535,0.039885,0.099732,0.874063,0.316709,4.0,8.0,0.994498,0.998758,0.988462,2.981718,0.484818,0.984007,0.405680,1.874505,3.0,1,1628,1705,77,3.433987,3.433987
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.365147,0.269472,0.221796,0.055430,0.392586,0.365147,0.0,9.0,0.000000,0.000000,0.000000,0.000000,0.996004,0.986181,0.983695,2.965880,0.0,1,1864,1952,88,2.298351,2.298351
2,0.527566,-1.510556,1.498903,0.137903,-0.957198,0.802047,2.441271,-1.826873,-2.086776,-1.725259,-0.975042,0.990319,0.495875,-1.821820,2.314325,1.171428,-0.295950,-2.536932,0.561701,1.311181,1.016990,2.515146,1.519219,-1.199506,0.399765,0.149002,-1.620842,-1.061517,-0.670159,-1.629803,-0.145572,-3.111086,-0.209722,0.287282,-0.889338,-1.291739,-0.057710,-1.274380,-2.490561,-0.031008,3.078424,-1.261173,0.525886,2.090457,1.480583,-2.677032,-1.590860,-2.423737,-1.109303,-0.921814,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.697569,0.651373,0.577294,0.102652,0.172420,0.697569,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.0,0.0,0.997263,0.996814,0.990159,2.984235,0.000000,0.000000,0.000000,0.000000,1.0,1,0,0,0,NaN,NaN
3,0.241789,-1.461544,-0.571965,0.051770,1.671800,-0.925241,-1.000506,0.147014,-2.080066,-0.262537,0.794321,-0.759372,1.713453,-1.130547,-0.407922,-2.496354,-0.767901,0.077382,0.731322,0.973145,-0.508279,-0.137868,-0.287884,-1.585357,0.120732,-2.239034,-1.965922,1.021036,-0.013576,-2.728281,0.682191,0.087695,0.773273,-1.560948,-1.544757,0.576463,-0.547967,-1.890046,-1.036790,0.309836,1.774340,-1.743814,0.684697,0.921820,1.642573,0.501846,1.311744,2.110542,-2.255831,-1.111799,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [6]:
test = test_X.drop(columns=['likes'])
test.shape

(12008, 371)

In [7]:
cat_params = {
    'depth' : 6,
    'learning_rate' : 0.1, 
    'eval_metric' :'RMSE', 
    'random_seed' :0,
    'num_boost_round': 10000,
}

In [8]:
categorical_features = ['LE_art_series_id', 'LE_copyright_holder', 'LE_acquisition_method',
                        'LE_principal_maker', 'LE_principal_or_first_maker',]

In [9]:
def train_catboost(X, y, params: dict, verbose=100):

    cv = KFold(n_splits=n_fold, shuffle=True, random_state=0)
    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv.split(X)): 
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
        print('x_train', x_train.shape)
        print('x_valid', x_valid.shape)
        
        train_pool = Pool(data=x_train, 
                                        label=y_train, 
                                        cat_features=categorical_features)
        validate_pool = Pool(data=x_valid, 
                                            label=y_valid, 
                                            cat_features=categorical_features)
        cat_model = CatBoostRegressor(**params)
        cat_model.fit(train_pool, eval_set=validate_pool, 
                                  verbose=verbose, 
                                  use_best_model=True,
                                  early_stopping_rounds=verbose)
        y_pred = cat_model.predict(validate_pool)
        
        oof_pred[idx_valid] = y_pred
        models.append(cat_model)
        print('Fold {} Score: {:.4f}'.format(i, np.sqrt(mean_squared_error(y_valid, y_pred))))

    score = np.sqrt(mean_squared_error(y, oof_pred))
    print('FINISHED \ whole score: {:.4f}'.format(score))
    return oof_pred, models, score

In [10]:
def train_pred(train_df, y, test_df):
    oof, models, score = train_catboost(train_df, y, cat_params)
    
    pred_list = []
    for model in models:
            test_pool = Pool(data=test_df, cat_features=categorical_features)
            pred = model.predict(test_pool)
#             pred = np.expm1(pred)
            pred_list.append(pred)
            print(pred.mean())
            
    pred = np.mean(pred_list, axis=0)
    return pred, score, models

In [11]:
def feat_imp(model):
    fi = model.feature_importance()
    fn = model.feature_name()
    df_feature_importance = pd.DataFrame({'name':fn, 'imp':fi})
    df_feature_importance.sort_values('imp', inplace=True)
    return df_feature_importance

def feature_importance(models):
    fi = pd.DataFrame(columns=['name'])
    for i, model in enumerate(models):
        fi_tmp = feat_imp(model)
        colname = 'imp_{}'.format(i)
        fi_tmp.rename(columns={'imp': colname}, inplace=True)
        fi = pd.merge(fi, fi_tmp, on=['name'], how='outer')
    fi['sum'] = fi.sum(axis=1)
    return fi.sort_values(['sum'], ascending=False)

In [12]:
%%time

pred, score, models = train_pred(train_X, target, test)

x_train (9620, 371)
x_valid (2406, 371)
0:	learn: 1.6856093	test: 1.6835039	best: 1.6835039 (0)	total: 211ms	remaining: 35m 12s
100:	learn: 0.9670962	test: 1.0262481	best: 1.0261983 (99)	total: 8.82s	remaining: 14m 24s
200:	learn: 0.8497023	test: 1.0020773	best: 1.0020773 (200)	total: 19.9s	remaining: 16m 10s
300:	learn: 0.7625033	test: 0.9914976	best: 0.9912962 (297)	total: 31.7s	remaining: 17m 2s
400:	learn: 0.6890557	test: 0.9860438	best: 0.9859949 (399)	total: 40.6s	remaining: 16m 12s
500:	learn: 0.6309314	test: 0.9839135	best: 0.9834815 (493)	total: 50s	remaining: 15m 48s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9834815052
bestIteration = 493

Shrink model to first 494 iterations.
Fold 0 Score: 0.9835
x_train (9621, 371)
x_valid (2405, 371)
0:	learn: 1.6923251	test: 1.6528556	best: 1.6528556 (0)	total: 101ms	remaining: 16m 47s
100:	learn: 0.9664443	test: 1.0195353	best: 1.0195353 (100)	total: 8.2s	remaining: 13m 23s
200:	learn: 0.8432259	test: 0.9988671

In [13]:
# feature_importance(models).head(50)

In [14]:
def revert_to_real(y_log):
    _pred = np.expm1(y_log)
    _pred = np.where(_pred < 0, 0, _pred)
    return _pred

In [19]:
test_pred = revert_to_real(pred)
print('test_pred', len(test_pred))
test_X['likes'] = test_pred
df_test_pred = test_X[['likes']]
df_test_pred.shape

test_pred 12008


(12008, 1)

In [23]:
FILENAME = 'catboost_1.csv'
OUTPUT_DIR = '../output/'
filepath = os.path.join(OUTPUT_DIR, FILENAME)
df_test_pred.to_csv(filepath, index=False)
print(filepath)

../output/catboost_1.csv


In [24]:
df_test_pred

,likes
0,1.441758
1,27.342095
2,0.526315
3,4.255415
4,0.275440
...,...
12003,0.097977
12004,0.001696
12005,0.275179
12006,0.076593
